In [1]:
import numpy as np
from scqubits import Fluxonium
import matplotlib.pyplot as plt


def calculate_energy(flxs, EJ, EC, EL, cutoff=50):
    fluxonium = Fluxonium(EJ, EC, EL, flux=0.0, cutoff=cutoff, truncated_dim=10)
    spectrumData = fluxonium.get_spectrum_vs_paramvals(
        "flux", flxs, evals_count=10
    )

    return spectrumData.energy_table

# load data

In [2]:
import h5py
from scipy.ndimage import gaussian_filter1d

def load_data(file_path):
    with h5py.File(file_path, "r") as file:
        data = file["Data"]['Data']
        if data.shape[2] == 1: # 1D data,
            x_data = data[:, 0, 0][:]
            y_data = None
            z_data = data[:, 1, 0][:] + 1j * data[:, 2, 0][:]
        else:
            x_data = data[:, 0, 0][:]
            y_data = data[0, 1, :][:]
            z_data = data[:, 2, :][:] + 1j * data[:, 3, :][:]
    return z_data, x_data, y_data

def convert2max_contrast(Is: np.ndarray, Qs: np.ndarray):
    # calculate the covariance matrix
    cov = np.cov(Is, Qs)

    # calculate the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(cov)

    # sort the eigenvectors by decreasing eigenvalues
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # rotate the data
    data = np.vstack([Is, Qs])
    data_rot = np.dot(eigenvectors.T, data)

    return data_rot[0], data_rot[1]

spectrum, fpts, flxs = load_data("data/qub_flux_dep.hdf5")
# spectrum, fpts, flxs = load_data("data/I007_TwoTone_flux_005.hdf5")
fpts /= 1e9 # convert to GHz

sps =[]
for i in range(spectrum.shape[1]):
    sp, _ = convert2max_contrast(spectrum[:, i].real, spectrum[:, i].imag)
    sps.append(sp)
spectrum = np.array(sps).T

if flxs[0] > flxs[-1]: # Ensure that the fluxes are in increasing
    flxs = flxs[::-1]
    spectrum = spectrum[:, ::-1]
if fpts[0] > fpts[-1]: # Ensure that the frequencies are in increasing
    fpts = fpts[::-1]
    spectrum = spectrum[::-1, :]

s_spectrum = spectrum.copy()
s_spectrum = gaussian_filter1d(s_spectrum, 1, axis=0)
s_spectrum = s_spectrum - np.median(s_spectrum, axis=0, keepdims=True)
s_spectrum = s_spectrum - np.median(s_spectrum, axis=1, keepdims=True)
s_spectrum = np.abs(s_spectrum)

norm_factor = np.std(s_spectrum, axis=0)
threshold = 0*1.5*np.mean(norm_factor)
norm_factor = np.where(norm_factor > threshold, norm_factor, threshold)
s_spectrum /= norm_factor

In [ ]:
%matplotlib widget
fig, ax = plt.subplots()
ax.imshow(s_spectrum, aspect='auto', origin='lower', extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))
background = fig.canvas.copy_from_bbox(ax.bbox)
cflx = (flxs[0] + flxs[-1]) / 2
eflx = flxs[-5]
line1 = ax.axvline(x=cflx, color='r', linestyle='--', picker=5)
line2 = ax.axvline(x=eflx, color='b', linestyle='--', picker=5)

picked = None
def onpick(event):
    global picked
    # check mouse press is on the line
    if event.mouseevent.name != 'button_press_event':
        return

    picked = event.artist if picked is None else None

def onmove(event):
    global picked
    if picked is None or event.inaxes != ax:
        return

    new_x = event.xdata
    # don't let the lines too close
    other_line = line2 if picked is line1 else line1
    other_x = other_line.get_xdata()[0]
    min_dist = 0.1 * (flxs[-1] - flxs[0])
    if new_x > other_x and new_x - other_x < min_dist:
        new_x = other_x + min_dist
    elif new_x < other_x and other_x - new_x < min_dist:
        new_x = other_x - min_dist
    if new_x > flxs[-1]:
        new_x = flxs[-1]
    elif new_x < flxs[0]:
        new_x = flxs[0]

    # only redraw the lines
    fig.canvas.restore_region(background)
    picked.set_xdata([new_x])
    ax.draw_artist(picked)
    ax.draw_artist(other_line)

fig.canvas.mpl_connect('pick_event', onpick)
fig.canvas.mpl_connect('motion_notify_event', onmove)

In [4]:
plt.close(fig)
center = line1.get_xdata()[0]
halfp = line2.get_xdata()[0] - center

cflxs = (flxs - center) / (2*halfp) + 0.5

In [ ]:
%matplotlib inline
plt.figure()
plt.imshow(s_spectrum, aspect='auto', origin='lower', extent=(cflxs[0], cflxs[-1], fpts[0], fpts[-1]))
plt.axvline(0.5, color='r')
plt.axvline(1.0, color='r')
plt.show()

In [6]:
flxs = cflxs

In [7]:
from scipy.ndimage import gaussian_filter1d

def spectrum_analyze(flxs, fpts, amps, ratio):
    # use guassian filter to smooth the spectrum
    amps = gaussian_filter1d(amps, 1, axis=0)
    amps = amps - np.median(amps, axis=0, keepdims=True)
    amps = amps - np.median(amps, axis=1, keepdims=True)
    amps = np.abs(amps)

    norm_factor = np.std(amps, axis=0)
    threshold = 1.5*np.mean(norm_factor)
    norm_factor = np.where(norm_factor > threshold, norm_factor, threshold)
    amps /= norm_factor # (len(flxs), len(fpts))

    # find peaks
    max_ids = np.argmax(amps, axis=0)
    maxs = amps[max_ids, np.arange(amps.shape[1])]  # (len(flxs),)

    # select points with large contrast
    max_masks = maxs >= ratio  # (len(flxs),)
    s_flxs = flxs[max_masks]
    s_fpts = fpts[max_ids][max_masks]

    return s_flxs, s_fpts

def remove_close_points(flxs, fpts, dist_ratio):
    # remove some close points
    mask = np.ones(len(flxs), dtype=bool)
    t_d2 = np.sqrt((flxs[-1] - flxs[0])**2 + (fpts[-1] - fpts[0])**2) * dist_ratio
    prev = 0
    for i in range(1, len(flxs)):
        d_flx = flxs[i] - flxs[prev]
        d_fs = fpts[i] - fpts[prev]
        d2 = np.sqrt(d_flx**2 + d_fs**2)
        if d2 < t_d2:
            mask[i] = False
        else:
            prev = i

    return flxs[mask], fpts[mask]

s_flxs, s_fpts = spectrum_analyze(flxs, fpts, spectrum, 4.2)
s_flxs, s_fpts = remove_close_points(s_flxs, s_fpts, 0.02)

In [ ]:
# interative remove points
%matplotlib widget
fig, ax = plt.subplots()
ax.imshow(s_spectrum, aspect='auto', origin='lower', extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))
background = fig.canvas.copy_from_bbox(ax.bbox)
colors = np.array(['r' for _ in range(len(s_flxs))])
collects = ax.scatter(s_flxs, s_fpts, color=colors, s=3, picker=5)

def onpick(event):
    global colors
    # check mouse press is on the point
    if event.mouseevent.name != 'button_press_event':
        return

    ind = event.ind[0]
    colors[ind] = 'k' if colors[ind] == 'r' else 'r'
    collects.set_color(colors)
    fig.canvas.restore_region(background)
    ax.draw_artist(collects)


fig.canvas.mpl_connect('pick_event', onpick)

In [9]:
plt.close(fig)
s_flxs = s_flxs[colors == 'r']
s_fpts = s_fpts[colors == 'r']

In [ ]:
%matplotlib inline
plt.figure()
plt.imshow(s_spectrum, aspect='auto', origin='lower', extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))
plt.scatter(s_flxs, s_fpts, color='r', s=3)
plt.show()

In [18]:
cavity_f = 5.8847
sample_f = 6.88128
transitions = [(0, 1), (0, 2), (1, 3)]
sideband = []
mirror = [(0, 3), (0, 2)]
def energy2transition(energies):
    fs = []
    labels = []
    fs.append(cavity_f*np.ones_like(energies[:, 0]))
    labels.append("cavity")
    for i, j in transitions:
        fs.append(energies[:, j] - energies[:, i])
        labels.append(f"{i} -> {j}")
    for i, j in sideband:
        freq = energies[:, j] - energies[:, i]
        fs.append(np.abs(freq - cavity_f))
        labels.append(f"{i} -> {j} side")
    for i, j in mirror:
        freq = energies[:, j] - energies[:, i]
        fs.append(2*sample_f - freq)
        labels.append(f"{i} -> {j} mirror")
    fs.append((2*sample_f - cavity_f)*np.ones_like(energies[:, 0]))
    labels.append("cavity mirror")

    return np.array(fs).T, labels

In [ ]:
import h5py as h5

with h5.File("database/fluxonium_1.h5", "r") as file:
    h_flxs = file["flxs"][:]  # type: ignore
    h_params = file["params"][:]  # type: ignore
    h_energies = file["energies"][:]  # type: ignore


def dist_to_curve(energies, s_fs):
    fs, _ = energy2transition(energies)

    dist = np.abs(fs - s_fs[:, None]) # (n, m)
    dist = np.nanmin(dist, axis=1) # (n, )

    return np.nansum(dist)

# find the closest index in energy to s_fs
sf_flxs = np.mod(s_flxs, 1.0)
d2 = (h_flxs[:, None] - sf_flxs[None, :])**2 # (m, m')
idxs = np.argmin(d2, axis=0) # (m', )

best_params = None
best_energy = None
best_dist = float("inf")
for i in range(h_params.shape[0]):
    energy = h_energies[i, idxs]

    dist = dist_to_curve(energy, s_fpts)
    if dist < best_dist:
        best_dist = dist
        best_energy = h_energies[i]
        best_params = h_params[i]
print(best_params)


In [ ]:
f_energies = calculate_energy(flxs, *best_params, cutoff=60)
# f_energies = calculate_energy(flxs, 6.31,0.935,1.175, cutoff=60)

In [ ]:
plt.imshow(s_spectrum, aspect="auto", origin="lower", extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))

fs, labels = energy2transition(f_energies)
for i, label in enumerate(labels):
    plt.plot(flxs, fs[:, i], label=label)

plt.scatter(s_flxs, s_fpts, c='r', s=3)
plt.ylim(fpts[0], fpts[-1])
plt.xlim(flxs[0], flxs[-1])
plt.legend()

# Scipy

In [152]:
EJb = (3.0, 6.5)
ECb = (0.3, 2.5)
ELb = (0.5, 3.5)
# EJb = (1.0, 3.0)
# ECb = (0.5, 2.5)
# ELb = (0.1, 0.5)

In [153]:
from scipy.optimize import minimize

# transitions = [(0, 1), (0, 3)]
def fit_spectrum(flxs, fpts, params):
    def loss_func(params):
        energies = calculate_energy(flxs, *params, cutoff=50)

        fs = []
        for i, j in transitions:
            fs.append(energies[:, j] - energies[:, i])
        for i, j in sideband:
            freq = energies[:, j] - energies[:, i]
            fs.append(np.abs(freq - cavity_f))
        fs = np.stack(fs, axis=1) # (n, m)
        dist = np.abs(fpts[:, None] - fs) # (n, m)
        loss_fs = np.nanmin(dist, axis=1) # (n,)

        return np.nansum(loss_fs)

    # aim to find current curve
    res = minimize(
        loss_func,
        params,
        bounds=(EJb, ECb, ELb),
        method="L-BFGS-B",
        options={"maxfun": 100},
    )

    return res.x

In [ ]:
# fit the spectrumData
sp_params = fit_spectrum(s_flxs, s_fpts, params=best_params)

# print the results
print("Fitted params:", *sp_params)

In [155]:
# sp_params = (
#     5.50,
#     1.12,
#     1.07
# )

In [ ]:
f_energies = calculate_energy(flxs, *sp_params, 60)

In [ ]:
# plt.pcolormesh(flxs, fpts, spectrum)
plt.imshow(s_spectrum, aspect='auto', origin='lower', extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))

# transitions = [(0, 1), (1, 2)]
for i, j in transitions:
    plt.plot(flxs, f_energies[:, j] - f_energies[:, i], label=f"{i}-{j}")
for i, j in sideband:
    freq = f_energies[:, j] - f_energies[:, i]
    plt.plot(flxs, np.abs(freq - cavity_f), label=f"{i}-{j} side")

plt.scatter(s_flxs, s_fpts, color="red", s=3)
plt.ylim(fpts[0], fpts[-1])
plt.legend()